# Assignment 3 - Navie Bayes UniversalBank
- Course: 64060 - Machine Learning and Data Analytics
- Datasource - UniversalBank.csv



In [49]:
# Imports
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import CategoricalNB

In [50]:
from google.colab import files

# Upload the CSV file
print("Please upload your UniversalBank.csv file:")
uploaded = files.upload()

# Load the data
df = pd.read_csv('UniversalBank.csv')
print("Data loaded successfully!")
print(f"Dataset shape: {df.shape}")

Please upload your UniversalBank.csv file:


Saving UniversalBank.csv to UniversalBank (1).csv
Data loaded successfully!
Dataset shape: (5000, 14)


In [51]:
# filtered dataframe
df_filtered = df[['Online', 'CreditCard', 'Personal Loan']]
df_filtered.rename(columns={'CreditCard': 'CC', 'Personal Loan': 'Loan'}, inplace=True)
# Display the first few rows
display(df_filtered.head())

/tmp/ipython-input-3519312235.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered.rename(columns={'CreditCard': 'CC', 'Personal Loan': 'Loan'}, inplace=True)


,Online,CC,Loan
0,0,0,0
1,0,0,0
2,0,0,0
3,0,0,0
4,0,1,0


In [52]:
# Define features-target
X = df_filtered[['Online', 'CC']]
y = df_filtered['Loan']

# 60-40 split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.4, random_state=1)

# combine training data back for ease of use
train_df = pd.concat([X_train, y_train], axis=1)

print(f"Training set shape: {train_df.shape}")
print(f"Validation set shape: {X_val.shape}")

Training set shape: (3000, 3)
Validation set shape: (2000, 2)


# Part A

In [53]:
train_df['count'] = 1

# Melt the dataframe to transform it into a long format
melted_df = pd.melt(train_df, id_vars=['CC', 'Loan', 'Online'], value_vars=['count'])

# Create the pivot table as required
pivot_A = melted_df.pivot_table(index=['CC', 'Loan'], columns='Online', values='value', aggfunc='sum', fill_value=0)
print(pivot_A)

Online     0     1
CC Loan           
0  0     792  1117
   1      73   126
1  0     327   477
   1      39    49


# Part B

In [54]:
# Extract the required counts
# Count(Loan=1, CC=1, Online=1)
count_accept = pivot_A.loc[(1, 1), 1]
# Count(Loan=0, CC=1, Online=1)
count_reject = pivot_A.loc[(1, 0), 1]
# Total customers with CC=1 and Online=1
total_conditional = count_accept + count_reject
# Calculate the probability P(Loan=1 | CC=1, Online=1)
prob_B = count_accept / total_conditional

print(f"P(Loan=1 | CC=1, Online=1) = {count_accept} / {total_conditional} = {prob_B:.4f}")
print("\n")

P(Loan=1 | CC=1, Online=1) = 49 / 526 = 0.0932




# Part C

In [55]:
# Create a pivot table for Loan as a function of Online
pivot_C_online = melted_df.pivot_table(index='Loan', columns='Online', values='value', aggfunc='sum', fill_value=0)
print(pivot_C_online)
print("-" * 20)
print("\n")

# Create a pivot table for Loan as a function of CC
pivot_C_cc = melted_df.pivot_table(index='Loan', columns='CC', values='value', aggfunc='sum', fill_value=0)
print(pivot_C_cc)
print("\n")

Online     0     1
Loan              
0       1119  1594
1        112   175
--------------------


CC       0    1
Loan           
0     1909  804
1      199   88




# Part D

In [56]:
total_train = len(train_df)
total_loan1 = train_df['Loan'].sum()
total_loan0 = total_train - total_loan1

# i. P(CC=1 | Loan=1)
p_cc1_given_loan1 = pivot_C_cc.loc[1, 1] / total_loan1

# ii. P(Online=1 | Loan=1)
p_online1_given_loan1 = pivot_C_online.loc[1, 1] / total_loan1

# iii. P(Loan=1)
p_loan1 = total_loan1 / total_train

# iv. P(CC=1 | Loan=0)
p_cc1_given_loan0 = pivot_C_cc.loc[0, 1] / total_loan0

# v. P(Online=1 | Loan=0)
p_online1_given_loan0 = pivot_C_online.loc[0, 1] / total_loan0

# vi. P(Loan=0)
p_loan0 = total_loan0 / total_train

# Print all to 4dp
print(f"i.   P(CC=1 | Loan=1) = {p_cc1_given_loan1:.4f}")
print(f"ii.  P(Online=1 | Loan=1) = {p_online1_given_loan1:.4f}")
print(f"iii. P(Loan=1) = {p_loan1:.4f}")
print(f"iv.  P(CC=1 | Loan=0) = {p_cc1_given_loan0:.4f}")
print(f"v.   P(Online=1 | Loan=0) = {p_online1_given_loan0:.4f}")
print(f"vi.  P(Loan=0) = {p_loan0:.4f}")
print("\n")

i.   P(CC=1 | Loan=1) = 0.3066
ii.  P(Online=1 | Loan=1) = 0.6098
iii. P(Loan=1) = 0.0957
iv.  P(CC=1 | Loan=0) = 0.2964
v.   P(Online=1 | Loan=0) = 0.5875
vi.  P(Loan=0) = 0.9043




# Part E

In [57]:
# p_cc1_given_loan1 * p_online1_given_loan1 * p_loan1
first = p_cc1_given_loan1 * p_online1_given_loan1 * p_loan1

# p_cc1_given_loan0 * p_online_given_loan0 * p_loan0
second = p_cc1_given_loan0 * p_online1_given_loan0 * p_loan0

total_conditional = first + second

# first / first + second
prob_E = first / (first + second)

print(f"Part E. P(Loan=1 | CC=1, Online=1) =  {prob_E}")

Part E. P(Loan=1 | CC=1, Online=1) =  0.10200430617247218


# Part F


In [58]:
print(f"The probability from the pivot table (Part B) is {prob_B:.4f}.")
print(f"The error on the Part B probability is {abs(p_loan1 - prob_B) / p_loan1 * 100:.4f}%")
print("\n")

print(f"The probability from the Naive Bayes calculation (Part E) is {prob_E:.4f}.")
print(f"The error on the Part E probability is {abs(p_loan1 - prob_E) / p_loan1 * 100:.4f}%")
print("\n")

print(f"The acutal is {p_loan1:.4f}.")
print("\nThe estimate from the pivot table (Part B) is the more accurate")
print("estimate for the training data itself. This is because it is the")
print("actual observed proportion in the data. The Naive Bayes result is")
print("an estimate based on the assumption of conditional independence")
print("between 'CC' and 'Online', which may not be perfectly true in the data.")

The probability from the pivot table (Part B) is 0.0932.
The error on the Part B probability is 2.6245%


The probability from the Naive Bayes calculation (Part E) is 0.1020.
The error on the Part E probability is 6.6247%


The acutal is 0.0957.

The estimate from the pivot table (Part B) is the more accurate
estimate for the training data itself. This is because it is the
actual observed proportion in the data. The Naive Bayes result is
an estimate based on the assumption of conditional independence
between 'CC' and 'Online', which may not be perfectly true in the data.


# Part G

In [59]:
print("Need to calculate the following")
print("1. Prior Probabilities: P(Loan=1) and P(Loan=0).")
print("2. Conditional Probabilities: P(CC=1|Loan=1), P(Online=1|Loan=1), etc.")
model = CategoricalNB()
model.fit(X_train, y_train)

# Define the customer case: [Online=1, CC=1]
customer_case = [[1, 1]]
predicted_probabilities = model.predict_proba(customer_case)
prob_G_model = predicted_probabilities[0, 1]
print("\n")
print("Model training and prediction complete.")

# --- Comparison ---
print(f"The model's predicted probability is: {prob_G_model:.4f}")
print(f"The manually calculated probability (Part E) was: {prob_E:.4f}")
print("The results match up to 2dp")


print("Getting a warning below - IGNORE!\n\n\n\n\n")


Need to calculate the following
1. Prior Probabilities: P(Loan=1) and P(Loan=0).
2. Conditional Probabilities: P(CC=1|Loan=1), P(Online=1|Loan=1), etc.


Model training and prediction complete.
The model's predicted probability is: 0.1023
The manually calculated probability (Part E) was: 0.1020
The results match up to 2dp
Getting a warning below - IGNORE!







/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but CategoricalNB was fitted with feature names
  warnings.warn(
